## NOTES:
* embedded is code to calculate each star school rank breakdown of absences percentages
* embedded is code to extract school_ward_int and school_cluster_int columns

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [78]:
data = pd.read_csv('../data/cleaned/school_df_v2.csv')

<br><br>
## Added in columns to look at Cluster and Ward as integer values

In [79]:
import re as re

def extract_number(text):
    num = re.findall(r'[0-9]+',text)
    return " ".join(num)

data['school_ward_int'] = data['school_ward'].apply(lambda x: extract_number(x))
data['school_cluster_int'] = data['school_cluster'].apply(lambda x: extract_number(x))

In [80]:
data.columns

Index(['school_sector', 'school_code', 'school_name', 'school_grade_range',
       'school_enrollment_SY1819', 'school_star_score_SY1819',
       'school_star_rating_SY1819', 'school_capacity_SY1819',
       'school_unfilled_seats_SY1819', 'school_latitude', 'school_longitude',
       'school_cluster', 'school_ward', 'school_grade_band',
       'school_enrollment_SY1718', 'school_star_score_SY1718',
       'school_star_rating_SY1718', 'school_capacity_SY1718',
       'school_unfilled_seats_SY1718',
       'attnd_count_truancy_aged_students_SY1819', 'attnd_absence_1-5_SY1819',
       'attnd_absence_6-10_SY1819', 'attnd_absence_11-20_SY1819',
       'attnd_absence_20+_SY1819', 'attnd_count_truancy_aged_students_SY1718',
       'attnd_absence_1-5_SY1718', 'attnd_absence_6-10_SY1718',
       'attnd_absence_11-20_SY1718', 'attnd_absence_20+_SY1718',
       'school_budgeted_amount_FY16', 'school_budgeted_enrollment_FY16',
       'school_budgeted_amount_FY17', 'school_budgeted_enrollment_FY17

In [82]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 35 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   school_sector                             116 non-null    object 
 1   school_code                               116 non-null    int64  
 2   school_name                               116 non-null    object 
 3   school_grade_range                        116 non-null    object 
 4   school_enrollment_SY1819                  116 non-null    float64
 5   school_star_score_SY1819                  109 non-null    float64
 6   school_star_rating_SY1819                 109 non-null    float64
 7   school_capacity_SY1819                    108 non-null    float64
 8   school_unfilled_seats_SY1819              108 non-null    float64
 9   school_latitude                           116 non-null    float64
 10  school_longitude                      

In [83]:
X = data.drop(columns=['school_ward', 'school_cluster', 'school_sector', 'school_name', 'school_grade_band', 'school_grade_range'])

In [84]:
X.shape

(116, 29)

In [91]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 29 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   school_code                               116 non-null    int64  
 1   school_enrollment_SY1819                  116 non-null    float64
 2   school_star_score_SY1819                  116 non-null    float64
 3   school_star_rating_SY1819                 116 non-null    float64
 4   school_capacity_SY1819                    116 non-null    float64
 5   school_unfilled_seats_SY1819              116 non-null    float64
 6   school_latitude                           116 non-null    float64
 7   school_longitude                          116 non-null    float64
 8   school_enrollment_SY1718                  116 non-null    float64
 9   school_star_score_SY1718                  116 non-null    float64
 10  school_star_rating_SY1718             

In [107]:
X = X.replace([np.inf, -np.inf], np.nan)
X = X.replace(np.nan, -1)
X = X.replace('<10', 5)
X = X.replace('n<10', 5)

In [108]:
X = X.apply(pd.to_numeric)

In [109]:
X.shape

(116, 29)

In [110]:
from sklearn.preprocessing import StandardScaler

X_scaled = StandardScaler().fit_transform(X)

In [111]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

dbscan=DBSCAN(eps=2.3, min_samples=4)
dbscan.fit(X_scaled)

DBSCAN(eps=2.3, min_samples=4)

In [112]:
set(dbscan.labels_)

{-1, 0, 1}

In [113]:
silhouette_score(X_scaled, dbscan.labels_)

-0.012640214205919535

In [116]:
def find_best_silhouette(df): 
    """select best eps and min_samples for a DBSCAN
    Args:
        df (pandas DataFrame): data to cluster
    Returns:
        None
    """
    max_score=-1     
    ss = StandardScaler()
    df_scaled = ss.fit_transform(df)

    for eps in np.linspace(.2, 5, 50):
        for minsamples in range(2, len(df/2)):
            dbscan = DBSCAN(eps=eps, min_samples=minsamples)
            dbscan.fit(df_scaled)
            if len(set(dbscan.labels_)) > 1:
                score = silhouette_score(df_scaled, dbscan.labels_)
                nclusters = len(set(dbscan.labels_))
                if score > max_score:
                    max_score = score
                    best_eps = eps
                    best_minsamples = minsamples
                    best_clusters = nclusters
    print(f'Best silhouette score was {round(max_score, 2)}')
    print(f'Best eps was {round(best_eps, 2)}')
    print(f'Best min_samples was {best_minsamples}.')
    print(f'Model found {best_clusters} clusters.')
    return

In [117]:
find_best_silhouette(X_scaled)

Best silhouette score was 0.56
Best eps was 5.0
Best min_samples was 2.
Model found 2 clusters.


In [119]:
X.columns

Index(['school_code', 'school_enrollment_SY1819', 'school_star_score_SY1819',
       'school_star_rating_SY1819', 'school_capacity_SY1819',
       'school_unfilled_seats_SY1819', 'school_latitude', 'school_longitude',
       'school_enrollment_SY1718', 'school_star_score_SY1718',
       'school_star_rating_SY1718', 'school_capacity_SY1718',
       'school_unfilled_seats_SY1718',
       'attnd_count_truancy_aged_students_SY1819', 'attnd_absence_1-5_SY1819',
       'attnd_absence_6-10_SY1819', 'attnd_absence_11-20_SY1819',
       'attnd_absence_20+_SY1819', 'attnd_count_truancy_aged_students_SY1718',
       'attnd_absence_1-5_SY1718', 'attnd_absence_6-10_SY1718',
       'attnd_absence_11-20_SY1718', 'attnd_absence_20+_SY1718',
       'school_budgeted_amount_FY16', 'school_budgeted_enrollment_FY16',
       'school_budgeted_amount_FY17', 'school_budgeted_enrollment_FY17',
       'school_ward_int', 'school_cluster_int'],
      dtype='object')

In [121]:
cluster_columns = ['school_code', 'school_star_score_SY1819', 'school_enrollment_SY1819', 'school_star_rating_SY1819', 'school_capacity_SY1819','attnd_absence_20+_SY1819', 'school_budgeted_amount_FY17','school_ward_int', 'school_cluster_int']

In [124]:
A = X[cluster_columns]

In [127]:
A_scaled = StandardScaler().fit_transform(A)
find_best_silhouette(A_scaled)

Best silhouette score was 0.57
Best eps was 4.41
Best min_samples was 5.
Model found 2 clusters.


<br><br>
## Generate Values and Summary Report for Absence Percentages Breakdown per Star Rating
* current code is worded for every star (1-5) for SY1718

In [ ]:
for i in range(1,6):
    
    print(f'SY1718 Star Rank of {i}: Summary')
    
    star_rank_df = data[data['school_star_rating_SY1718'] == i]

    count_enrolled = star_rank_df['school_enrollment_SY1718'].sum()
    count_any = star_rank_df['students_with_absences_SY1718'].sum()
    count_0 = star_rank_df['school_enrollment_SY1718'].sum()-star_rank_df['students_with_absences_SY1718'].sum()
    count_1_5 = star_rank_df['attnd_absence_1-5_SY1718'].sum()
    count_6_10 = star_rank_df['attnd_absence_6-10_SY1718'].sum()
    count_11_20 = star_rank_df['attnd_absence_11-20_SY1718'].sum()
    count_20plus = star_rank_df['attnd_absence_20+_SY1718'].sum()
    
    pct_0 = (star_rank_df['school_enrollment_SY1718'].sum()-star_rank_df['students_with_absences_SY1718'].sum())/star_rank_df['school_enrollment_SY1718'].sum()
    pct_1_5 = star_rank_df['attnd_absence_1-5_SY1718'].sum()/star_rank_df['school_enrollment_SY1718'].sum()
    pct_6_10 = star_rank_df['attnd_absence_6-10_SY1718'].sum()/star_rank_df['school_enrollment_SY1718'].sum()
    pct_11_20 = star_rank_df['attnd_absence_11-20_SY1718'].sum()/star_rank_df['school_enrollment_SY1718'].sum()
    pct_20plus = star_rank_df['attnd_absence_20+_SY1718'].sum()/star_rank_df['school_enrollment_SY1718'].sum()
    
    print('count of students in rank ', i, ' school:', count_enrolled)
    print('count of students in rank ', i, ' school with any absences:', count_any)
    print('count of students in rank ', i, ' school with no absences:', count_0)
    print('percent of rank ', i, ' school students with no absences:', pct_0)
    print('count of students in rank ', i, ' school with 1-5 absences:', count_1_5)
    print('percent of rank ', i, ' school students with 1-5 absences:', pct_1_5)
    print('count of students in rank ', i, ' school with 6-10 absences:', count_6_10)
    print('percent of rank ', i, ' school students with 6-10 absences:', pct_6_10)
    print('count of students in rank ', i, ' school with 11-20 absences:', count_11_20)
    print('percent of rank ', i, ' school students with 11-20 absences:', pct_11_20)
    print('count of students in rank ', i, ' school with 20+ absences:', count_20plus)
    print('percent of rank ', i, ' school students with 20+ absences:', pct_20plus)
    print('\n')

<br><br>
## Generate Pie Charts for Absence Percentages Breakdown per Star Rating
* current code is worded for every star (1-5) for SY1718

In [ ]:
def func(pct, values): 
    absolute = int(pct / 100.*np.sum(values)) 
    return "{:.1f}%".format(pct, absolute)

for i in range(1,6):

    star_rank_df = data[data['school_star_rating_SY1718'] == i]

    pct_0 = (star_rank_df['school_enrollment_SY1718'].sum()-star_rank_df['students_with_absences_SY1718'].sum())/star_rank_df['school_enrollment_SY1718'].sum()
    pct_1_5 = star_rank_df['attnd_absence_1-5_SY1718'].sum()/star_rank_df['school_enrollment_SY1718'].sum()
    pct_6_10 = star_rank_df['attnd_absence_6-10_SY1718'].sum()/star_rank_df['school_enrollment_SY1718'].sum()
    pct_11_20 = star_rank_df['attnd_absence_11-20_SY1718'].sum()/star_rank_df['school_enrollment_SY1718'].sum()
    pct_20plus = star_rank_df['attnd_absence_20+_SY1718'].sum()/star_rank_df['school_enrollment_SY1718'].sum()

    pcts = np.array([pct_0,pct_1_5,pct_6_10,pct_11_20,pct_20plus])
    pct_labels = ["0", "1-5", "6-10", "11-20", "20+"]

    wedges, text, autotexts = plt.pie(pcts, labels = pct_labels, autopct = lambda pct: func(pct, pcts))

    plt.setp(autotexts, size = 10, color='white')
    plt.title(f'Star Rank {i} - SY17/18')
    plt.show()

<br><br>
## Code to create dataframe of only static school identification values (code, ward, cluster, etc.) and columns pertaining to SY1819 and FY17

In [ ]:
data.drop(columns = data.filter(regex='SY1718$',axis=1)).drop(columns = data.filter(regex='FY16$',axis=1))

<br><br>
## Code to create dataframe of only static school identification values (code, ward, cluster, etc.) and columns pertaining to SY1718 and FY16

In [ ]:
data.drop(columns = data.filter(regex='SY1819$',axis=1)).drop(columns = data.filter(regex='FY17$',axis=1))